In [1]:
import numpy as np
import pandas as pd

Definimos las variables

In [2]:
N = 5 # Número de meses
r = np.zeros([N+1, N+1]) # Arbol Binomial
Q = np.zeros([N+1, N+1]) # Precios Arrow-Debreu
theta = np.zeros(N+1) # Drift
dt = 1/12 # Delta t
sigma = 0.002952452925930132 # Volatilidad

In [3]:
r[0][0] = 0.04761 # Tasa inicial
Q[0][0] = 1 # Precio inicial

# Leer factores de descuento
with open('df.txt', 'r') as archivo:
    discount_factors = archivo.read()
    discount_factors = discount_factors.split(',')
    for i in range(len(discount_factors)):
        discount_factors[i] = float(discount_factors[i])
    P = np.array(discount_factors)

theta[0] = 1 - P[0]*2 + np.sqrt(1 + 4*(P[1]**2)*(dt**3)*(sigma**2))
theta[0] = theta[0]/(2*P[1]*dt)
theta[0] = (1/dt)*(theta[0] - r[0][0])

In [4]:
theta[0]

-0.5713192764552077

Tasas nivel 1

In [5]:
r[1][1] = r[0][0] + theta[0]*dt + sigma*np.sqrt(dt)
r[0][1] = r[0][0] + theta[0]*dt - sigma*np.sqrt(dt)

Funciones para despejar theta

In [6]:
# Factor de descuento
def disc_factor(r: float, dt: float)->float:
    return 1 / (1 + dt*r)

def ab_ident(theta: float, j: int, P: list, Q : list, r: list, dt: float, sigma: float)->float:
    
    value = 0.5 * Q[0][j] * disc_factor(r[0][j] + theta*dt - sigma*np.sqrt(dt), dt)
    value += 0.5 * Q[j][j] * disc_factor(r[j][j] + theta*dt + sigma*np.sqrt(dt), dt)

    middle_nodes = [(Q[i][j] + Q[i+1][j])*disc_factor(r[i+1][j] + theta*dt - sigma*np.sqrt(dt), dt) 
                    for i in range(j-1)]
    value += 0.5 * sum(middle_nodes)
    value -= P[j+1]

    return value

def ab_ident_deriv(theta: float, j: int, P: list, Q : list, r: list, dt: float, sigma: float)->float:

    value = (-0.5) * (dt**2) * Q[0][j] * (disc_factor(r[0][j] + theta*dt - sigma*np.sqrt(dt), dt)**2)
    value -= 0.5 * (dt**2) * Q[j][j] * (disc_factor(r[j][j] + theta*dt + sigma*np.sqrt(dt), dt)**2)

    middle_nodes = [(Q[i][j] + Q[i+1][j])*(disc_factor(r[i+1][j] + theta*dt - sigma*np.sqrt(dt), dt)**2) 
                    for i in range(j-1)]
    value -= 0.5 * (dt**2) * sum(middle_nodes)

    return value


Función Newton-Rahpson

In [7]:
def newton_raphson(j: int, P: list, Q : list, r: list, dt: float, sigma: float)->float:
    theta = 0
    for i in range(10000):
        theta = theta - (ab_ident(theta, j, P, Q, r, dt, sigma)/ab_ident_deriv(theta, j, P, Q, r, dt, sigma))
    return theta

In [8]:
Q[0][1] = disc_factor(r[0][1], dt) * Q[0][0]
Q[1][1] = disc_factor(r[1][1], dt) * Q[0][0]

In [9]:
newton_raphson(1, P, Q, r, dt, sigma)

1.1400036150145074

In [10]:
ab_ident(1.140361500133895, 1, P, Q, r, dt, sigma)

-2.4464189573025763e-06